In [103]:
import os

import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [104]:
def recoverImage(img, t, A=np.array([1,1,1])):
    t_0 = 0.1
    J = np.zeros(img.shape, dtype='float64')

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            J[i,j] = ((img[i,j] - A) / max(t[i,j], t_0)) + A
            
    return J

def haze2dehazed(haze_paths, dMap_paths, output_dir):

    os.makedirs(output_dir, exist_ok=True)
    
    for img_path, dm_path in tqdm(zip(haze_paths, dMap_paths), total=len(haze_paths)):
        
        img_name = os.path.split(img_path)[1]
        
        dehaze_img_name = os.path.join(output_dir, img_name)
#         if os.path.isfile(dehaze_img_name):
#             continue
            
        img = cv2.imread(img_path)/255
        dmap = cv2.imread(dm_path)/255
       
        # the generated depth map's shape could be slightly off compared to the original image so fix that
        dmap = cv2.resize(dmap, (img.shape[1], img.shape[0]))

        dmap = cv2.cvtColor(dmap.astype('float32'), cv2.COLOR_BGR2GRAY)
        
#         tmap = np.clip(np.exp(-dmap), a_min=0, a_max=1)
        tmap = dmap
        dehaze_img = recoverImage(img, tmap)

        # write to disk
        cv2.imwrite(dehaze_img_name, dehaze_img*255)


In [105]:
dmap_dir = "dhazy/Middlebury_predictedDepthMap_refined"
hazy_dir = "dhazy/Middlebury_Hazy_reshaped"
output_dir = hazy_dir+"_dehazed"

In [108]:
dmap_paths = []
hazy_paths = []

for f in os.listdir(hazy_dir):
    hazy_path = os.path.join(hazy_dir, f)
    dmap_path = os.path.join(dmap_dir, f.replace("_Hazy", "_predDMap"))
    
    assert os.path.isfile(dmap_path) and os.path.isfile(hazy_path)
    dmap_paths.append(dmap_path)
    hazy_paths.append(hazy_path)  

In [109]:
print(hazy_paths[:5], dmap_paths[:5])

['dhazy/Middlebury_Hazy_reshaped/Vintage_Hazy.bmp', 'dhazy/Middlebury_Hazy_reshaped/Sword1_Hazy.bmp', 'dhazy/Middlebury_Hazy_reshaped/Bicycle1_Hazy.bmp', 'dhazy/Middlebury_Hazy_reshaped/Piano_Hazy.bmp', 'dhazy/Middlebury_Hazy_reshaped/Jadeplant_Hazy.bmp'] ['dhazy/Middlebury_predictedDepthMap_refined/Vintage_predDMap.bmp', 'dhazy/Middlebury_predictedDepthMap_refined/Sword1_predDMap.bmp', 'dhazy/Middlebury_predictedDepthMap_refined/Bicycle1_predDMap.bmp', 'dhazy/Middlebury_predictedDepthMap_refined/Piano_predDMap.bmp', 'dhazy/Middlebury_predictedDepthMap_refined/Jadeplant_predDMap.bmp']


In [110]:
haze2dehazed(hazy_paths, dmap_paths, output_dir)


100%|██████████| 23/23 [01:00<00:00,  2.65s/it]
